# JSON Agent

This notebook showcases an agent designed to interact with large JSON/dict objects. This is useful when you want to answer questions about a JSON blob that's too large to fit in the context window of an LLM. The agent is able to iteratively explore the blob to find what it needs to answer the user's question.

In the below example, we are using the OpenAPI spec for the OpenAI API, which you can find [here](https://github.com/openai/openai-openapi/blob/master/openapi.yaml).

First, we answer some questions about the API spec. Then, as a follow-up, we configure an agent to use the JSON agent as a tool, along with a requests toolkit, to make a request to the endpoint based on the API spec.

## Initialization

In [12]:
import os
import yaml

from langchain.agents import (
    AgentExecutor,
    initialize_agent,
    load_tools,
    Tool,
    ZeroShotAgent
)
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import RequestsWrapper
from langchain.tools.json.tool import JsonSpec
from langchain.tools.json.toolkit import JsonToolkit
from langchain.tools.requests.toolkit import RequestsToolkit

In [13]:
with open("../../../../notebooks/openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_toolkit = JsonToolkit(spec=JsonSpec(dict_=data, max_value_length=4000))

json_agent = ZeroShotAgent.as_json_agent(
    llm=OpenAI(temperature=0),
    toolkit=json_toolkit
)
json_agent_executor = AgentExecutor.from_agent_and_tools(
    agent=json_agent, tools=json_toolkit.get_tools(), verbose=True
)

## Example: getting the required POST parameters for a request

In [14]:
json_agent_executor.run("What are the required parameters in the request body to the /completions endpoint?")



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the paths key to see what endpoints exist
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/engines', '/engines/{engine_id}', '/completions', '/edits', '/images/generations', '/images/edits', '/images/variations', '/embeddings', '/engines/{engine_id}/search', '/files', '/files/{file_id}', '/files/{file_id}/content', '/answers', '/classifications', '/fine-tunes', '/fine-tunes/{fine_tune_id}', '/fine-tunes/{fine_tune_id}/cancel', '/fine-tunes/{fine_tune_id}/events', '/models', '/models/{model}', '/moderations']
Thought: I should look at the /completions endpoint to see what parameters are required
Action: json_spec_list_keys
Action Input: data["paths"]["/completions"]
Observation: ['post']
Thought: I should look at the post key to see what parameters are required
Actio

"The required parameters in the request body to the /completions endpoint are 'model'."

## Example: agent capable of analyzing OpenAPI spec and making requests

In [17]:
headers = {
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
}
request_toolkit = RequestsToolkit(requests_wrapper=RequestsWrapper(headers=headers))
tools = [
    *request_toolkit.get_tools(),
    Tool(
        name="json_explorer",
        func=json_agent_executor.run,
        description="""
        Can be used to answer questions about the openapi spec for the API. Always use this tool before trying to make a request. 
        Example inputs to this tool: 
            'What are the required query parameters for a GET request to the /bar endpoint?`
            'What are the required parameters in the request body for a POST request to the /foo endpoint?'
        Always give this tool a specific question.
        """
    )
]

prefix = """
You are an agent designed to answer questions by making web requests to an API given the openapi spec.

If the question does not seem related to the API, return I don't know. Do not make up an answer.
Only use information provided by the tools to construct your response.

First, find the base URL needed to make the request.

Second, find the relevant paths needed to answer the question. Take note that, sometimes, you might need to make more than one request to more than one path to answer the question.

Third, find the required parameters needed to make the request. For GET requests, these are usually URL parameters and for POST requests, these are request body parameters.

Fourth, make the requests needed to answer the question. Ensure that you are sending the correct parameters to the request by checking which parameters are required. For parameters with a fixed set of values, please use the spec to look at which values are allowed.

Use the exact parameter names as listed in the spec, do not make up any names or abbreviate the names of parameters.
If you get a not found error, ensure that you are using a path that actually exists in the spec.
"""
suffix = """Begin!"

Question: {input}
Thought: I should explore the spec to find the base url for the API.
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [19]:
agent_executor.run("Make a post request to openai /completions. The prompt should be 'tell me a joke.'")



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the servers key to see what the base url is
Action: json_spec_list_keys
Action Input: data["servers"][0]
Observation: ValueError('Value at path `data["servers"][0]` is not a dict, get the value directly.')
Thought: I should get the value of the servers key
Action: json_spec_get_value
Action Input: data["servers"][0]
Observation: {'url': 'https://api.openai.com/v1'}
Thought: I now know the base url for the API
Final Answer: The base url for the API is https://api.openai.com/v1

> Finished chain.

Observation: The base url for the API is https://api.openai.com/v1
Thought: I should find the path and parameters needed to make the request.
Action: json_explorer
Action Input: What are th

'"I really need a joke to cheer me up"'